# February 08, 2024 - Session Notes

##### 1. Math LLM Chain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
chat = ChatGoogleGenerativeAI(model="gemini-pro")

In [ ]:
from langchain.chains import LLMMathChain

In [ ]:
llm_math_model = LLMMathChain.from_llm(llm=chat)

In [ ]:
llm_math_model.invoke("What is 81 to the power of 7")


##### 2. Question - Answering Chains (4 types)

A Question-Answering in LangChain is a sequence of components that work together to answer questions based on a given context. The chain typically consists of a retriever component that identifies relevant documents from a corpus, a reader component that extracts information from those documents, and a generator component that synthesizes a response to the question. There are four different types of QA chain. Namely:
- load_qa chain
- RetrieveQA chain
- VectorStoreIndexCreator
- ConversationalRetrieval Chain

Let us look one by one in detail

(a) - `load_qa_chain`: This chain is used to load question-answering model and a vector store. The vector store contains the context documents that the model can use to answer questions. This chain is typically used as a building block for other chains.


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_openai import ChatOpenAI

In [ ]:
#chat = ChatGoogleGenerativeAI(model="gemini-pro",convert_system_message_to_human=True)
chat = ChatOpenAI()

In [ ]:
#embed_func = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embed_func = OpenAIEmbeddings()

In [ ]:
loader = PyPDFLoader("./Data/generative-ai-on-the-cusp-of-disruption-a-primer.pdf")
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)

In [ ]:
docs = text_splitter.split_documents(documents)

In [ ]:
GenAIPrimerDB = Chroma(embedding_function=embed_func, persist_directory="./GenAI_Primer")

In [ ]:
chain = load_qa_chain(llm=chat,chain_type="stuff",verbose=True)

In [ ]:
result = chain.invoke({"input_documents":docs, "question":"What are the Enterprise use cases of Generative AI?"})

In [ ]:
print (result["output_text"])

In [ ]:
print(result["input_documents"][1].page_content)

(b) - `RetrieveQA` chain: This chain is used to retrieve relevant documents from a vector store based on a given question, and then use a question-answering model to generate an answer from those documents. This chain is useful when you have a large corpus of documents and you want to find the most relevant ones to answer a question.

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
retriever = GenAIPrimerDB.as_retriever(search_type ="similarity", search_kwargs={'k':2})

In [ ]:
chain = RetrievalQA.from_chain_type(llm=chat, chain_type="stuff",retriever=retriever, return_source_documents=True)

In [ ]:
result=chain.invoke("What is Lab45?")

In [ ]:
print(result["result"])

(c) - `VectorStoreIndexCreator`: This chain is used to create a vector store from a set of documents. The vector store is a data structure that allows for efficient similarity search, enabling the retrieval of relevant documents based on a given question. This chain is typically used as a preprocessing step for other chains.

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
index = VectorstoreIndexCreator(text_splitter=text_splitter, embedding=embed_func, vectorstore_cls=Chroma).from_loaders(loaders=[loader])

In [ ]:
print (index.query ("What are the Enterprise use cases of Generative AI?"))